# Zeroshot vs. Keyword

Benchmarking our keyword approach against a zeroshot classifier

Findings: They find most of the same labels. Amoung the cc labeled our keyword search finds that do not exists in the zeroshot cc-labeled sampled, we see find that only 1.5/29 are not cc related. (gone over by manual revision)

In [2]:
import sys
sys.path.append('/home/gugy/Documents/political-ad-api/') 
import pandas as pd
import regex as re
from src.political_ads.Preprocessor import *
from src.political_ads.keyword_filter import *
from src.political_ads.zeroshot_text_preprocessing import *

In [3]:
#Code to read from our full dataset:
preprocess = Preprocessor()
df = preprocess.file_to_df('../data/all_politicians_aggregated.txt')

In [4]:
#zeroshot sample
zeroshot = pd.read_csv("unique_ads_preprocessed_labeled.csv")

In [5]:
# apply this to the dataframe
clean_data(zeroshot)
#apply it to df
zeroshot_cleaned = zeroshot_preprocess(zeroshot)
zeroshot_cleaned = zeroshot_cleaned[['zero_shot_label','clean_text']]

/home/gugy/Documents/political-ad-api/src/political_ads/zeroshot_text_preprocessing.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['clean_text'] = dataframe['ad_creative_body'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


As we see below, there are still duplicates in the data as there appear to still be variances of the same ads just with diff geographical reach and impressions (perhaps). Below we an example with the ones that that have the same content (ad_creative_body), but different impr_geo_agg (geography)

In [6]:
zeroshot[zeroshot.clean_text.duplicated(keep=False)].sort_values(by='clean_text')


,ad_creative_body,impr_geo_agg,spend_agg,impr_agg,page_id,page_name,ad_creation_time,party,emotions,ANGRY,...,AMUSED,AFRAID,HAPPY,INSPIRED,SAD,DONT_CARE,zero_shot_label,sent_length,language,clean_text
54938,a dark money gop pac just launched a $4 milli...,{'Washington': 500},49.5,499.5,1354622541311687,Dr. Kim Schrier,2021-05-19,Democrat,"{'AFRAID': 0.094005332573729, 'AMUSED': 0.1414...",0.129725,...,0.141458,0.094005,0.103749,0.157567,0.098531,0.133501,not_classified,long,en,a dark money gop pac just launched a $4 milli...
54939,a dark money gop pac just launched a $4 milli...,"{'Oregon': 5, 'Idaho': 5, 'Tennessee': 5, 'Cal...",49.5,499.5,1354622541311687,Dr. Kim Schrier,2021-05-19,Democrat,"{'AFRAID': 0.094005332573729, 'AMUSED': 0.1414...",0.129725,...,0.141458,0.094005,0.103749,0.157567,0.098531,0.133501,not_classified,long,en,a dark money gop pac just launched a $4 milli...
54940,a dark money gop pac just launched a $4 milli...,"{'Massachusetts': 5, 'New Mexico': 2, 'Minneso...",49.5,499.5,1354622541311687,Dr. Kim Schrier,2021-05-19,Democrat,"{'AFRAID': 0.094005332573729, 'AMUSED': 0.1414...",0.129725,...,0.141458,0.094005,0.103749,0.157567,0.098531,0.133501,not_classified,long,en,a dark money gop pac just launched a $4 milli...
54941,a dark money gop pac just launched a $4 milli...,"{'Michigan': 4, 'Vermont': 2, 'Texas': 24, 'Wa...",49.5,499.5,1354622541311687,Dr. Kim Schrier,2021-05-19,Democrat,"{'AFRAID': 0.094005332573729, 'AMUSED': 0.1414...",0.129725,...,0.141458,0.094005,0.103749,0.157567,0.098531,0.133501,not_classified,long,en,a dark money gop pac just launched a $4 milli...
49232,a dark money gop pac just launched a $4 milli...,"{'Wyoming': 2, 'Wisconsin': 40, 'Illinois': 29...",49.5,1499.5,1354622541311687,Dr. Kim Schrier,2021-05-19,Democrat,"{'AFRAID': 0.094005332573729, 'AMUSED': 0.1414...",0.129725,...,0.141458,0.094005,0.103749,0.157567,0.098531,0.133501,not_classified,long,en,a dark money gop pac just launched a $4 milli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52424,"“trump, gop states ask appeals court to kill t...","{'New York': 995, 'Connecticut': 4}",99.0,999.0,359153387814647,Antonio Delgado,2019-09-30,Democrat,"{'AFRAID': 0.10772768060856265, 'AMUSED': 0.13...",0.141637,...,0.131392,0.107728,0.103672,0.153453,0.104106,0.119508,not_classified,long,en,"“trump, gop states ask appeals court to kill t..."
33303,⏰ deadline approaching ⏰ \n\nwashington republ...,"{'Arkansas': 26, 'Washington': 257, 'New York'...",149.5,9499.5,135002490472474,Conor Lamb,2019-12-20,Democrat,"{'AFRAID': 0.10362943224384256, 'AMUSED': 0.14...",0.123352,...,0.142761,0.103629,0.116039,0.141456,0.109936,0.128870,not_classified,long,en,⏰ deadline approaching ⏰ \n\nwashington republ...
49800,⏰ deadline approaching ⏰ \n\nwashington republ...,"{'Tennessee': 2, 'Minnesota': 4, 'Florida': 13...",49.5,1499.5,135002490472474,Conor Lamb,2019-12-20,Democrat,"{'AFRAID': 0.10362943224384256, 'AMUSED': 0.14...",0.123352,...,0.142761,0.103629,0.116039,0.141456,0.109936,0.128870,not_classified,long,en,⏰ deadline approaching ⏰ \n\nwashington republ...
46033,⏰ deadline approaching ⏰ \n\nwashington republ...,"{'North Dakota': 1, 'Alabama': 9, 'Kansas': 10...",49.5,2499.5,135002490472474,Conor Lamb,2019-12-20,Democrat,"{'AFRAID': 0.10362943224384256, 'AMUSED': 0.14...",0.123352,...,0.142761,0.103629,0.116039,0.141456,0.109936,0.128870,not_classified,long,en,⏰ deadline approaching ⏰ \n\nwashington republ...


In [7]:
#zeroshot labels related to climate change and environment:
climate_zeroshot_labels_1 = ['climate', 'environment',
       'climate, economy', 'climate, immigration',
       'environment, education', 'climate, education',
       'climate, inequality',
       'environment, economy']

# zeroshot labels related to climate change only
climate_zeroshot_labels_2 = ['climate',
       'climate, economy', 'climate, immigration', 'climate, education',
       'climate, inequality']

climate_zeroshot_labels = climate_zeroshot_labels_2

zeroshot_cleaned_dub = zeroshot_cleaned[zeroshot_cleaned['zero_shot_label'].isin(climate_zeroshot_labels) == True]

print("zer0 before dub removal", len(zeroshot_cleaned_dub))

zeroshot_cleaned = zeroshot_cleaned_dub[zeroshot_cleaned_dub.clean_text.duplicated()==False]

print("zer0 after dub removal", len(zeroshot_cleaned))


zer0 before dub removal 1260
zer0 after dub removal 1030


In [8]:
zeroshot_cleaned

,zero_shot_label,clean_text
24,climate,unbelievable after stacey abrams helped chang...
124,climate,named the hill s top climate nerd businessman ...
184,climate,rep alexandria ocasio cortez and i just introd...
254,climate,take any of our big problems health care clima...
332,climate,50 000 responses missing 11 59 p m tonight mi...
...,...,...
57432,climate,the trump administration announced its plans t...
57520,climate,i challenged trump s epa nominee andrew wheele...
57551,climate,yesterday my daughter climbed the half dome in...
57766,climate,we don t have time to waste if we re going to ...


Loading the full data set


In [9]:
# Adding preprocessing steps to the dataset
NaNFree = df[df.ad_creative_body.isna()==False]
clean_data(NaNFree)
full_set = zeroshot_preprocess(NaNFree)

#Adding keyword climate labels
c_filter = Filter()
c_filter.add_climate_label(full_set)


/home/gugy/Documents/political-ad-api/src/political_ads/zeroshot_text_preprocessing.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['clean_text'] = dataframe['ad_creative_body'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
/home/gugy/Documents/political-ad-api/src/political_ads/zeroshot_text_preprocessing.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['clean_text'] = dataframe['ad_creative_body'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


Joining the zeroshot_cleaned data set onto the full set. This way we map it back to the ~600,000 sample and ensure that 

In [10]:
full = full_set.merge(zeroshot_cleaned, left_on='clean_text', right_on='clean_text', how='left')

In [11]:
full[full.zero_shot_label.isna()==False]

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,clean_text,keyword_label,zero_shot_label
5744,2020-01-26,I’m proposing a tax on the top 0.1% — the Jeff...,49.5,499.5,NaN,NaN,38471053686,Elizabeth Warren,"WARREN FOR PRESIDENT, INC.",603945340405334,0,99,0,999,i m proposing a tax on the top 0 1 the jeff be...,Climate,climate
5745,2020-01-26,I’m proposing a tax on the top 0.1% — the Jeff...,49.5,499.5,NaN,NaN,38471053686,Elizabeth Warren,"WARREN FOR PRESIDENT, INC.",480434179313755,0,99,0,999,i m proposing a tax on the top 0 1 the jeff be...,Climate,climate
5747,2020-01-26,I’m proposing a tax on the top 0.1% — the Jeff...,49.5,499.5,NaN,NaN,38471053686,Elizabeth Warren,"WARREN FOR PRESIDENT, INC.",205612473901399,0,99,0,999,i m proposing a tax on the top 0 1 the jeff be...,Climate,climate
5751,2020-01-26,I’m proposing a tax on the top 0.1% — the Jeff...,49.5,499.5,NaN,NaN,38471053686,Elizabeth Warren,"WARREN FOR PRESIDENT, INC.",177115937004752,0,99,0,999,i m proposing a tax on the top 0 1 the jeff be...,Climate,climate
5752,2020-01-26,I’m proposing a tax on the top 0.1% — the Jeff...,49.5,499.5,NaN,NaN,38471053686,Elizabeth Warren,"WARREN FOR PRESIDENT, INC.",178652719899689,0,99,0,999,i m proposing a tax on the top 0 1 the jeff be...,Climate,climate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597672,2019-05-06,Climate change is one of the most pressing iss...,49.5,499.5,NaN,NaN,280704452367167,Jason Crow for Congress,Jason Crow for Congress,455789565194363,0,99,0,999,climate change is one of the most pressing iss...,Climate,climate
597677,2019-05-06,Climate change is one of the most pressing iss...,49.5,499.5,"[{'percentage': '1', 'region': 'Colorado'}]","[{'percentage': '0.026087', 'age': '45-54', 'g...",280704452367167,Jason Crow,Jason Crow for Congress,394531874725343,0,99,0,999,climate change is one of the most pressing iss...,Climate,climate
598018,2020-09-18,We deserve better in the 170th. We deserve som...,49.5,2499.5,"[{'percentage': '0.084771', 'region': 'New Jer...","[{'percentage': '0.087151', 'age': '45-54', 'g...",174750213023790,"Mike Doyle for PA Representative, 170th District",Friends to Elect Mike Doyle,372997463705253,0,99,2000,2999,we deserve better in the 170th we deserve some...,Climate,climate
598020,2020-09-17,We deserve better in the 170th. We deserve som...,49.5,2499.5,"[{'percentage': '0.158678', 'region': 'New Jer...","[{'percentage': '0.000463', 'age': '45-54', 'g...",174750213023790,"Mike Doyle for PA Representative, 170th District",Friends to Elect Mike Doyle,1194045847638527,0,99,2000,2999,we deserve better in the 170th we deserve some...,Climate,climate


Now to the analysis and comparison in performance of the two approaches...

In [12]:

true_positives = full[(full.keyword_label=='Climate') & (full.zero_shot_label.isna()==False)]
false_positives = full[(full.keyword_label=='Climate') & (full.zero_shot_label.isna()==True)]
true_negatives = full[(full.keyword_label=='Non-climate') & (full.zero_shot_label.isna()==True)]
false_negatives = full[(full.keyword_label=='Non-climate') & (full.zero_shot_label.isna()==False)]

In [13]:
total_climate_zero = len(full[full.zero_shot_label.isna()==False])
total_climate_keyword = len(full[full.keyword_label=='Climate'])

In [33]:
print("zeroshot climate", total_climate_zero)
print("keyword climate", total_climate_keyword)

print("\n\nNOTE: This is when using the climate tags for the zeroshot classifier", climate_zeroshot_labels)

zeroshot climate 18893
keyword climate 19176


NOTE: This is when using the climate tags for the zeroshot classifier ['climate', 'climate, economy', 'climate, immigration', 'climate, education', 'climate, inequality']


In [15]:
#Benchmarked against the Zeroshot.
print("True positives", len(true_positives), len(true_positives)/len(full[full.zero_shot_label.isna()==False]))

print("False positives",len(false_positives), len(false_positives)/len(full[full.zero_shot_label.isna()==False]))

print("True negatives", len(true_negatives), len(true_negatives)/len(full[full.zero_shot_label.isna()==True]))

print("False negatives", len(false_negatives), len(false_negatives)/len(full[full.zero_shot_label.isna()==False]))

True positives 18876 0.999100195839729
False positives 300 0.01587889694595882
True negatives 578866 0.9994820137922461
False negatives 17 0.0008998041602709998


In [16]:
zero_vs_keyword = {'keyword':total_climate_keyword, 'zeroshot':total_climate_zero, 'overlap': len(true_positives)}
zero_vs_keyword

{'keyword': 19176, 'zeroshot': 18893, 'overlap': 18876}

In [17]:
false_pos_texts = false_positives['clean_text'].unique()
false_neg_texts = false_negatives.clean_text.unique()

#### False Positives

In [43]:
false_pos_texts

false_pos = pd.read_excel("false_positives_climate_tags.xlsx").drop(columns=['Unnamed: 0'])

print("unique texts in the false positives sample (i.e., climate change labeled ads from keyword, that don't appear in the zeroshot climate labeled sample:", len(false_pos))

climate_change_related_false_pos = false_pos['Climate Change'].sum()/len(false_pos)
climate_change_related_false_pos

unique texts in the false positives sample (i.e., climate change labeled ads from keyword, that don't appear in the zeroshot climate labeled sample: 29


0.9482758620689655

Almost all (~95%) are actually about climate change and zeroshot mistakenly labeled them climate-related.

#### False Negatives 

In [31]:
print("Unique texts in the false neg sample (i.e., what zeroshot labels climate related):", len(false_neg_texts), ", none of which are actually climate related and include topics such as \"cold weather\", homelessness, business climate")
print("\nCONTENT:")
false_neg_texts

Unique texts in the false neg sample (i.e., what zeroshot labels climate related): 4 , none of which are actually climate related and include topics such as "cold weather", homelessness, business climate

CONTENT:


array(['no one who is experiencing or at risk of homelessness should have to face colorado s severe weather conditions the uncertainty of where they will sleep at night or the dangers of being unsheltered it s on us to make sure that all denver residents have access to the services that they need to live safe and fulfilling lives that s why i m working hard to secure federal funds to support non profit organizations addressing the homelessness crisis in denver ',
       'saturday is caucus day this is the most important election of our lives and we need your support now make a plan to caucus today ',
       'texas came in on top as the state with the best business climate in business facilities 16th annual rankings report the strength of texas economy and conservative values will be a guiding force as america continues to reopen and rebuild ',
       'the weather is getting colder but nothing will stop our team from getting out there to earn every vote we can today we spoke to voters a

In [19]:
false_negatives.groupby("zero_shot_label").count()

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,clean_text,keyword_label
zero_shot_label,,,,,,,,,,,,,,,,
climate,16,16,16,16,15,15,16,16,16,16,16,16,16,16,16,16
"climate, economy",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
